# Import Testing Framework

This notebook demonstrates how to test live download progress events from the Civitai Download Daemon backend. We will use Python's `unittest` framework and the `websockets` and `httpx` libraries to:

- Start a real (or mock) download
- Connect to the WebSocket endpoint
- Assert that progress events are received during the download

Let's start by importing the required libraries.

In [26]:
import unittest
import asyncio
# import websockets
from websockets.sync.client import connect
import httpx
import time

In [27]:
import nest_asyncio
nest_asyncio.apply()

# Define the Test Scenario

We want to verify that when a download is started via the API, the backend emits real-time progress events over the WebSocket. For this test, we'll:

- Start a download of a small file (or a mock endpoint)
- Connect to the WebSocket endpoint
- Collect and display progress events

The test will pass if at least one progress event is received during the download.

In [28]:
class TestLiveDownloadProgress(unittest.TestCase):
    import uuid
    WS_URL = "ws://localhost:8000/ws/downloads"
    API_URL = "http://localhost:8000/api/download"
    TEST_FILE_URL = "https://nbg1-speed.hetzner.com/1GB.bin"  # Aanbevolen, niet deprecated

    def collect_progress_events(self, model_id, filename):
        import json
        events = []
        headers = [("Authorization", "Bearer testtoken")]
        msg_count = 0
        try:
            with connect(self.WS_URL, additional_headers=headers) as ws:
                start = time.time()
                while time.time() - start < 30:  # Maximaal 30 seconden wachten
                    try:
                        msg = ws.recv()
                        msg_count += 1
                        print('WS message:', msg)  # Debug: print all received messages
                        print('WS message type:', type(msg), 'repr:', repr(msg))
                        try:
                            event_obj = json.loads(msg)
                        except Exception as e:
                            print(f"Failed to parse WS message as JSON: {e}")
                            continue
                        print('Parsed event:', event_obj)
                        events.append(event_obj)
                        event_type = event_obj.get('event')
                        if event_type == 'download_finished':
                            print("Breaking on first download_finished event.")
                            return events  # Break on first download_finished event, regardless of model_id/filename
                    except Exception as e:
                        print(f"WebSocket recv failed: {type(e).__name__}: {e}")
                        break
        except Exception as e:
            print(f"WebSocket connection failed: {type(e).__name__}: {e}")
        if msg_count == 0:
            print("WARNING: No messages received from WebSocket!")
        return events

    def test_live_progress(self):
        # Use a unique model_id and filename for each test run
        import uuid
        model_id = f"test_live_progress_{uuid.uuid4().hex[:8]}"
        filename = f"testfile_{uuid.uuid4().hex[:8]}.bin"
        payload = {
            "model_id": model_id,
            "model_version_id": "v1",
            "url": self.TEST_FILE_URL,
            "filename": filename,
            "model_type": "other"
        }
        with httpx.Client() as client:
            resp = client.post(self.API_URL, json=payload, headers={"Authorization": "Bearer testtoken"})
            print(f"API response: {resp.status_code} {resp.text}")
            if resp.status_code != 200:
                print("Download API call failed, skipping progress test.")
                return

        events = self.collect_progress_events(model_id, filename)
        if len(events) == 0:
            print("No progress events received!")
        else:
            print(f"Received {len(events)} events.")
            print("Last event:", events[-1])
            print("Test completed and exited on first download_finished event.")


# Run the Tests and Display Results

The following cell will run the test and display the output. Make sure your backend is running and accessible at `localhost:8000` before running this test.

In [29]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_live_progress (__main__.TestLiveDownloadProgress.test_live_progress) ... 

API response: 200 {"status":"queued","item":{"model_id":"test_live_progress_6e618aff","url":"https://nbg1-speed.hetzner.com/1GB.bin","filename":"testfile_ac94d63d.bin","sha256":null,"priority":0,"retries":0,"model_type":"other","model_version_id":"v1","base_model":null,"after_download_hook":{}}}
WS message: {"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename": "testfile_ac94d63d.bin", "progress": 0, "downloaded": 16062, "total": 1048576000}}
WS message type: <class 'str'> repr: '{"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename": "testfile_ac94d63d.bin", "progress": 0, "downloaded": 16062, "total": 1048576000}}'
Parsed event: {'event': 'download_progress', 'data': {'model_id': 'test_live_progress_6e618aff', 'filename': 'testfile_ac94d63d.bin', 'progress': 0, 'downloaded': 16062, 'total': 1048576000}}
WS message: {"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename

ok

----------------------------------------------------------------------
Ran 1 test in 14.376s

OK


WS message: {"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename": "testfile_ac94d63d.bin", "progress": 100, "downloaded": 1048576000, "total": 1048576000}}
WS message type: <class 'str'> repr: '{"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename": "testfile_ac94d63d.bin", "progress": 100, "downloaded": 1048576000, "total": 1048576000}}'
Parsed event: {'event': 'download_progress', 'data': {'model_id': 'test_live_progress_6e618aff', 'filename': 'testfile_ac94d63d.bin', 'progress': 100, 'downloaded': 1048576000, 'total': 1048576000}}
WS message: {"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename": "testfile_ac94d63d.bin", "progress": 100, "downloaded": 1048576000, "total": 1048576000}}
WS message type: <class 'str'> repr: '{"event": "download_progress", "data": {"model_id": "test_live_progress_6e618aff", "filename": "testfile_ac94d63d.bin", "progress": 100, "downloa

In [30]:
# Enable testtoken override for authentication (must be run before the test)
import httpx
resp = httpx.post("http://localhost:8000/test/enable-testtoken")
print(resp.status_code, resp.text)
if resp.status_code == 200:
    print("Testtoken override enabled.")
else:
    print(f"Failed to enable testtoken override: {resp.status_code} {resp.text}")

200 {"status":"testtoken enabled"}
Testtoken override enabled.


In [31]:
# Debug: Check CIVITAI_TEST_AUTH environment variable from the backend
import httpx
resp = httpx.get("http://localhost:8000/test/env?key=CIVITAI_TEST_AUTH")
print("Backend reports CIVITAI_TEST_AUTH:", resp.status_code, resp.text)

Backend reports CIVITAI_TEST_AUTH: 200 {"key":"CIVITAI_TEST_AUTH","value":"1"}
